# Skipgram Analysis on Mozart Piano Sonatas

Compute skipgrams over skipgrams

## Setup

Add worker processes (local and remote).
The code must be in the same directory on all nodes, otherwise addprocs needs to be told where to start.

In [1]:
# remotes go first, otherwise a two-way ssh connection is needed
#addprocs([("remote-name-or-addr", :auto)])
addprocs([("128.179.134.225", :auto)])

addprocs(3) # leave one core free locally

3-element Array{Int64,1}:
 6
 7
 8

In [2]:
# loading code on all workers

# musicology library
# Pkg.clone("https://github.com/DCMLab/DigitalMusicology.jl.git")
using DigitalMusicology

# load the schema skipgrams code
@everywhere include("skipgrams.jl")

In [3]:
# set the corpus directory (must be the same on all nodes, otherwise don't use @everywhere)
#@everywhere DigitalMusicology.usekern("/path/to/kern/dir");
@everywhere DigitalMusicology.usekern("/home/chfin/Uni/phd/data/csapp/mozart-piano-sonatas/");

## Parameters

In [10]:
# number of voices per stage
voices = 2

# number of stages
stages = 3

2

In [7]:
# samplin in the first pass (groups of notes)
p1 = 1.0
# p1 = 0.1 # 3x4 only!!!

# sampling in the second pass (sequences of groups)
p2 = 1.0 # 2x2

1.0

In [5]:
# list pieces with their respective bar lengths
pieces = map(p -> (p, Unsims.piecebarlen(p)), allpieces())

69-element Array{Tuple{String,Rational{Int64}},1}:
 ("sonata01-1", 1//1)
 ("sonata01-2", 3//4)
 ("sonata01-3", 1//2)
 ("sonata02-1", 3//4)
 ("sonata02-2", 3//4)
 ("sonata02-3", 3//8)
 ("sonata03-1", 1//2)
 ("sonata03-2", 3//8)
 ("sonata03-3", 1//1)
 ("sonata04-1", 1//1)
 ("sonata04-2", 3//4)
 ("sonata04-3", 1//2)
 ("sonata05-1", 3//4)
 ⋮                   
 ("sonata14-1", 1//1)
 ("sonata14-2", 1//1)
 ("sonata14-3", 3//4)
 ("sonata15-1", 1//1)
 ("sonata15-2", 3//4)
 ("sonata15-3", 1//2)
 ("sonata16-1", 3//4)
 ("sonata16-2", 1//1)
 ("sonata16-3", 1//1)
 ("sonata17-1", 3//4)
 ("sonata17-2", 3//4)
 ("sonata17-3", 1//2)

In [8]:
srand(111)

MersenneTwister(UInt32[0x0000006f], Base.dSFMT.DSFMT_state(Int32[-1742434217, 1072888606, -1808167675, 1073264677, -816900598, 1073279215, -1688324199, 1073349469, 966630521, 1072721743  …  -182202980, 1073352782, -336987111, 1073634920, 293846579, -2027139531, -1236354088, -20277153, 382, 0]), [1.74216, 1.88128, 1.19347, 1.1648, 1.92639, 1.24087, 1.55934, 1.33381, 1.3449, 1.76115  …  1.41883, 1.16047, 1.28335, 1.06074, 1.43403, 1.31577, 1.38478, 1.06707, 1.52229, 1.25502], 382)

## Enumerate Skipgrams

In [11]:
@time counts = Unsims.countpiecesschemasbars(pieces, voices, stages, p2, p1) # yes, p2, p1!

	From worker 8:	processing sonata03-1
	From worker 7:	processing sonata02-3
	From worker 6:	processing sonata02-2
	From worker 2:	processing sonata01-1
	From worker 7:	done with  sonata02-3
	From worker 7:	processing sonata03-2
	From worker 7:	done with  sonata03-2
	From worker 7:	processing sonata04-1
	From worker 6:	done with  sonata02-2
	From worker 6:	processing sonata04-2
	From worker 6:	done with  sonata04-2
	From worker 6:	processing sonata04-3
	From worker 6:	done with  sonata04-3
	From worker 6:	processing sonata05-1
	From worker 6:	done with  sonata05-1
	From worker 6:	processing sonata06-1
	From worker 8:	done with  sonata03-1
	From worker 8:	processing sonata06-3e
	From worker 8:	done with  sonata06-3e
	From worker 8:	processing sonata06-3f
	From worker 8:	done with  sonata06-3f
	From worker 8:	processing sonata06-3h
	From worker 8:	done with  sonata06-3h
	From worker 8:	processing sonata06-3j
	From worker 8:	done with  sonata06-3j
	From worker 8:	processing sonata06-3m
	Fr

Trie{Array{DigitalMusicology.Pitches.MidiPitch,1},Float64}(...)

In [16]:
# save computed skipgrams
fn = "counts_$(voices)x$(stages)_p1_$(p1)_p2_$(p2)_$(now()).jls"
open(f -> serialize(f, counts), joinpath("official_counts", fn), "w")

Execution times:

| n | s | p1  | p2  | time |
|---|---|-----|-----|------|
| 2 | 2 | 1.0 | 1.0 | 796s |